In [1]:
# ЯЧЕЙКА 1: Импорты
import os
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
from tqdm import tqdm

# Пути к директориям (при необходимости)
RMT_INFER_DIR = "/mnt/10tb/home/patsukevichan/borZoi/borzoi/examples/rmt_infer/"
BORZOI_BEDGRAPH_DIR = "/mnt/10tb/home/patsukevichan/borZoi/borzoi/examples/predicted_expression_by_chromosomes/"

print("Libraries imported. Paths set.")


Libraries imported. Paths set.


In [2]:
# ЯЧЕЙКА 2: Загрузка таблиц Алёны

# Пути к parquet-файлам
labels_parquet = os.path.join(RMT_INFER_DIR, "labels.parquet")
labels_transcripts_parquet = os.path.join(RMT_INFER_DIR, "labels_transcripts.parquet")
preds_2seg_parquet = os.path.join(RMT_INFER_DIR, "preds_transcripts_2seg.parquet")
preds_4seg_parquet = os.path.join(RMT_INFER_DIR, "preds_transcripts_4seg.parquet")
preds_8seg_parquet = os.path.join(RMT_INFER_DIR, "preds_transcripts_8seg.parquet")

# Читаем parquet с помощью pandas (или pyarrow)
labels_df = pd.read_parquet(labels_parquet)
labels_transcripts_df = pd.read_parquet(labels_transcripts_parquet)
preds_2seg_df = pd.read_parquet(preds_2seg_parquet)
preds_4seg_df = pd.read_parquet(preds_4seg_parquet)
preds_8seg_df = pd.read_parquet(preds_8seg_parquet)

# Для наглядности:
print("labels_df:")
display(labels_df.head())

print("labels_transcripts_df:")
display(labels_transcripts_df.head())

print("preds_2seg_df:")
display(preds_2seg_df.head())

print("preds_4seg_df:")
display(preds_4seg_df.head())

print("preds_8seg_df:")
display(preds_8seg_df.head())


labels_df:


,chrom_pred,preds,chrom,start,end,start_pred,end_pred,Transcript stable ID,description_id
0,chr10,-4.60517,chr10,72893738,72893859,72893735,72893739,ENST00000334011,ENCFF123KIW
1,chr10,-4.60517,chr10,72893738,72893859,72893739,72893743,ENST00000334011,ENCFF123KIW
2,chr10,-4.60517,chr10,72893738,72893859,72893743,72893749,ENST00000334011,ENCFF123KIW
3,chr10,-4.60517,chr10,72893738,72893859,72893749,72893752,ENST00000334011,ENCFF123KIW
4,chr10,-4.60517,chr10,72893738,72893859,72893752,72893758,ENST00000334011,ENCFF123KIW


labels_transcripts_df:


,Transcript stable ID,description_id,preds
0,ENST00000001008,ENCFF123KIW,4.037295
1,ENST00000002596,ENCFF123KIWr,4.281860
2,ENST00000002829,ENCFF123KIW,5.388129
3,ENST00000009180,ENCFF123KIW,7.723541
4,ENST00000009589,ENCFF123KIWr,8.866660


preds_2seg_df:


,Transcript stable ID,description_id,preds
0,ENST00000001008,ENCFF123KIW,0.570057
1,ENST00000002596,ENCFF123KIWr,-2.881656
2,ENST00000002829,ENCFF123KIW,1.192969
3,ENST00000009180,ENCFF123KIW,2.320119
4,ENST00000009589,ENCFF123KIWr,4.619201


preds_4seg_df:


,Transcript stable ID,description_id,preds
0,ENST00000001008,ENCFF123KIW,2.727709
1,ENST00000002596,ENCFF123KIWr,0.732307
2,ENST00000002829,ENCFF123KIW,2.426938
3,ENST00000009180,ENCFF123KIW,4.060136
4,ENST00000009589,ENCFF123KIWr,4.862711


preds_8seg_df:


,Transcript stable ID,description_id,preds
0,ENST00000001008,ENCFF123KIW,3.736615
1,ENST00000002596,ENCFF123KIWr,1.674160
2,ENST00000002829,ENCFF123KIW,0.736134
3,ENST00000009180,ENCFF123KIW,4.146430
4,ENST00000009589,ENCFF123KIWr,5.075033


In [3]:
# Очищаем labels_df, удаляя ненужные столбцы preds и оставляем только chrom, start, end, Transcript stable ID и description_id
labels_clear_df = labels_df[['chrom', 'start', 'end', 'Transcript stable ID', 'description_id']]

# Убираем дублирующиеся записи по 'Transcript stable ID' и оставляем уникальные
labels_clear_df = labels_clear_df.drop_duplicates(subset=['Transcript stable ID'])

# Выводим для проверки
print("Очищенные данные labels_clear_df:")
display(labels_clear_df.head())


Очищенные данные labels_clear_df:


,chrom,start,end,Transcript stable ID,description_id
0,chr10,72893738,72893859,ENST00000334011,ENCFF123KIW
379,chr10,73071294,73071357,ENST00000431293,ENCFF123KIW
425,chr10,73110454,73110567,ENST00000357321,ENCFF123KIW
772,chr10,73168118,73168386,ENST00000242505,ENCFF123KIW
1660,chr10,73247359,73247383,ENST00000440197,ENCFF123KIW


In [4]:
# ЯЧЕЙКА 4: Создание словаря сопоставлений description_id с файлом для bedGraph

# Создадим таблицу для соответствия description_id и имен файлов
experiment_mapping = {
    "ENCFF123KIW": "borzoi_rnaseq_fold3_ENCSR892LBU+_fw.bedGraph",
    "ENCFF123KIWr": "borzoi_rnaseq_fold3_ENCSR892LBU-_fw.bedGraph",
    "ENCFF784MDF": "borzoi_rnaseq_fold3_ENCSR357BYU+_fw.bedGraph",
    "ENCFF784MDFr": "borzoi_rnaseq_fold3_ENCSR357BYU-_fw.bedGraph",
    "ENCFF236XOK": "borzoi_rnaseq_fold3_ENCSR763OMY+_fw.bedGraph",
    "ENCFF236XOKr": "borzoi_rnaseq_fold3_ENCSR763OMY-_fw.bedGraph",
    "ENCFF781TTC": "borzoi_rnaseq_fold3_ENCSR071DYD+_fw.bedGraph",
    "ENCFF781TTCr": "borzoi_rnaseq_fold3_ENCSR071DYD-_fw.bedGraph",
    "ENCFF242BWW": "borzoi_rnaseq_fold3_ENCSR045GTF+_fw.bedGraph",
    "ENCFF242BWWr": "borzoi_rnaseq_fold3_ENCSR045GTF-_fw.bedGraph"
}

# Выводим для проверки
print("Словарь сопоставлений для description_id:")
for key, value in experiment_mapping.items():
    print(f"{key}: {value}")

Словарь сопоставлений для description_id:
ENCFF123KIW: borzoi_rnaseq_fold3_ENCSR892LBU+_fw.bedGraph
ENCFF123KIWr: borzoi_rnaseq_fold3_ENCSR892LBU-_fw.bedGraph
ENCFF784MDF: borzoi_rnaseq_fold3_ENCSR357BYU+_fw.bedGraph
ENCFF784MDFr: borzoi_rnaseq_fold3_ENCSR357BYU-_fw.bedGraph
ENCFF236XOK: borzoi_rnaseq_fold3_ENCSR763OMY+_fw.bedGraph
ENCFF236XOKr: borzoi_rnaseq_fold3_ENCSR763OMY-_fw.bedGraph
ENCFF781TTC: borzoi_rnaseq_fold3_ENCSR071DYD+_fw.bedGraph
ENCFF781TTCr: borzoi_rnaseq_fold3_ENCSR071DYD-_fw.bedGraph
ENCFF242BWW: borzoi_rnaseq_fold3_ENCSR045GTF+_fw.bedGraph
ENCFF242BWWr: borzoi_rnaseq_fold3_ENCSR045GTF-_fw.bedGraph


In [6]:
# ЯЧЕЙКА 5: Загрузка bedGraph файлов в оперативную память (fw вариант)

# Создадим список всех файлов для загрузки
bedgraph_files_fw = list(experiment_mapping.values())

# Загрузим все файлы в оперативную память
bedgraph_data = {}
for file_name in tqdm(bedgraph_files_fw, desc="Обработка файлов"):
    file_path = os.path.join(BORZOI_BEDGRAPH_DIR, file_name)
    # Читаем bedGraph данные
    if os.path.exists(file_path):
        bedgraph_data[file_name] = pd.read_csv(file_path, sep='\t', header=None, names=['chrom', 'start', 'end', 'value'])
    else:
        print(f"Файл не найден: {file_name}")

# Проверим, что данные загружены
print("Пример загруженных данных для файла:")
display(bedgraph_data[bedgraph_files_fw[0]].head())

Обработка файлов: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:19<00:00,  7.93s/it]

Пример загруженных данных для файла:


,chrom,start,end,value
0,chr1,143480137,143480169,0.010857
1,chr1,143480169,143480201,0.007740
2,chr1,143480201,143480233,0.007561
3,chr1,143480233,143480265,0.006142
4,chr1,143480265,143480297,0.005950


In [15]:
display(bedgraph_data[bedgraph_files_fw[0]]['value'].max())

4430.953125

In [7]:
# ЯЧЕЙКА 6: Вычисление среднего покрытия по экзонам для каждого транскрипта

# Создаем DataFrame для хранения результатов
preds_borzoi = []

# Проходим по всем строкам labels_transcripts_df
for _, row in tqdm(labels_transcripts_df.iterrows(), total=labels_transcripts_df.shape[0], desc="Обработка транскриптов"):
    transcript_id = row['Transcript stable ID']
    description_id = row['description_id']
    
    # Находим соответствующие start и end для транскрипта
    transcript_data = labels_clear_df[labels_clear_df['Transcript stable ID'] == transcript_id]
    start = transcript_data['start'].values[0]
    end = transcript_data['end'].values[0]
    
    # Получаем имя файла на основе description_id
    matching_file = experiment_mapping.get(description_id, None)
    if not matching_file:
        continue

    # Получаем данные для этого транскрипта из bedGraph
    bedgraph_df = bedgraph_data.get(matching_file, None)
    if bedgraph_df is None:
        continue
    
    # Фильтруем по нужным координатам (start, end)
    region_data = bedgraph_df[(bedgraph_df['start'] >= start) & (bedgraph_df['end'] <= end)]
    
    # Рассчитываем среднее покрытие
    mean_coverage = region_data['value'].mean() if not region_data.empty else np.nan
    
    # Добавляем результат в preds_borzoi
    preds_borzoi.append([transcript_id, description_id, mean_coverage])

# Создаем финальный DataFrame для предсказаний
preds_borzoi_df = pd.DataFrame(preds_borzoi, columns=['Transcript stable ID', 'description_id', 'preds'])

# Выводим результаты
display(preds_borzoi_df.head())

Обработка транскриптов: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22095/22095 [09:37<00:00, 38.23it/s]


,Transcript stable ID,description_id,preds
0,ENST00000001008,ENCFF123KIW,NaN
1,ENST00000002596,ENCFF123KIWr,0.911151
2,ENST00000002829,ENCFF123KIW,NaN
3,ENST00000009180,ENCFF123KIW,0.012894
4,ENST00000009589,ENCFF123KIWr,0.010816


In [ ]:
# Объединяем данные из preds_borzoi и метки из labels_transcripts_df

# Для начала получим только нужные столбцы из labels_transcripts_df
labels_transcripts_df_clean = labels_transcripts_df[['Transcript stable ID', 'description_id', 'preds']]

# Объединяем с preds_borzoi для получения нужных меток
merged_preds_borzoi = pd.merge(preds_borzoi_df, labels_transcripts_df_clean, on=['Transcript stable ID', 'description_id'], how='inner')

# Рассчитываем FPKM для Борзой (используем значения как метки)
# FPKM = (предсказание Борзой / длина транскрипта) * 10^9
merged_preds_borzoi['FPKM_borzoi'] = merged_preds_borzoi['preds'] / (merged_preds_borzoi['end'] - merged_preds_borzoi['start']) * 1e9

# Выводим часть DataFrame для проверки
display(merged_preds_borzoi.head())
